<a href="https://colab.research.google.com/github/Unilaw-colab/AutoVisit_SEO_Urls/blob/main/AutoVisit_SEO_URLs_ThayDoi_CauHinhMay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cập nhật danh sách gói và cài đặt các công cụ cần thiết
!apt-get update
!apt-get install -y wget unzip
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

# Cài đặt các thư viện Python cần thiết
!pip install --upgrade pip
!pip install --upgrade gspread oauth2client selenium webdriver-manager

# Import các thư viện cần thiết
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import random
import json
import os
from selenium.webdriver.common.by import By

# File lưu danh sách các liên kết đã click
CLICKED_LINKS_FILE = 'clicked_links.json'

# Hàm lưu các liên kết đã click
def save_clicked_links(clicked_links):
    with open(CLICKED_LINKS_FILE, 'w') as file:
        json.dump(clicked_links, file)
    print("Đã lưu danh sách các liên kết đã click.")

# Hàm nạp các liên kết đã click
def load_clicked_links():
    if os.path.exists(CLICKED_LINKS_FILE):
        with open(CLICKED_LINKS_FILE, 'r') as file:
            clicked_links = json.load(file)
        print("Đã nạp danh sách các liên kết đã click.")
        return set(clicked_links)
    else:
        print("Không tìm thấy file liên kết đã click, tạo danh sách mới.")
        return set()

# Hàm lấy dữ liệu từ Google Sheets
def get_urls_from_google_sheet(sheet_url):
    creds_file = 'google-sheets-key.json'
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_file, scope)
    client = gspread.authorize(creds)
    sheet = client.open_by_url(sheet_url).worksheet("Sheet4")
    data = sheet.get_all_records()
    urls = [row['Post Link'] for row in data if row.get('Post Link')]
    return urls

# Hàm cấu hình Selenium
def setup_driver():
    options = Options()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = "/usr/bin/google-chrome"
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    # Tạo trình duyệt
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Ẩn thuộc tính webdriver
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
            })
        """
    })
    return driver

# Hàm mô phỏng đọc bài viết (không click link)
def simulate_reading(driver, url, clicked_links, min_time=180, max_time=300):
    try:
        driver.get(url)
        time.sleep(random.uniform(3, 6))

        scroll_height = driver.execute_script("return document.body.scrollHeight")
        current_position = 0
        reading_time = random.uniform(min_time, max_time)

        start_time = time.time()
        while time.time() - start_time < reading_time:
            scroll_distance = random.randint(300, 700)
            current_position = min(current_position + scroll_distance, scroll_height)
            driver.execute_script(f"window.scrollTo(0, {current_position});")
            time.sleep(random.uniform(7, 10))
            if current_position >= scroll_height:
                break

        print(f"Đã mô phỏng đọc bài viết: {url}")
    except Exception as e:
        print(f"Lỗi khi truy cập URL {url}: {e}")

# Hàm reset runtime
def reset_runtime(driver):
    driver.quit()  # Đóng trình duyệt
    print("Đang reset runtime để thay đổi IP...")
    time.sleep(random.uniform(60, 90))  # Nghỉ trước khi khởi động lại

# Hàm chính
def main():
    sheet_url = "https://docs.google.com/spreadsheets/d/1KnarSqWMf9EFndYNlB4sdC2wvnKAlBz1rukv8w4WcnY/edit"
    print("Đang lấy danh sách URL từ Google Sheets (Sheet4)...")
    urls = get_urls_from_google_sheet(sheet_url)

    print(f"Đã lấy {len(urls)} URL: {urls}")
    if not urls:
        print("Không tìm thấy URL nào trong Google Sheets.")
        return

    # Nạp các liên kết đã click từ phiên trước
    clicked_links = load_clicked_links()

    print("Đang thiết lập Selenium...")
    driver = setup_driver()

    print("Đang truy cập các URL...")
    max_visits = 5
    url_visit_count = {url: 0 for url in urls}

    start_time = time.time()
    while any(count < max_visits for count in url_visit_count.values()):
        if time.time() - start_time > random.uniform(1800, 2700):  # Reset sau 30-45 phút
            reset_runtime(driver)
            driver = setup_driver()
            start_time = time.time()

        available_urls = [url for url, count in url_visit_count.items() if count < max_visits]
        url = random.choice(available_urls)

        if url in clicked_links:
            print(f"Bỏ qua URL đã click: {url}")
            continue

        url_visit_count[url] += 1
        simulate_reading(driver, url, clicked_links)

        time.sleep(random.uniform(3, 8))

    driver.quit()
    print("Quá trình hoàn tất.")

# Chạy chương trình
main()


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,331 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,226 kB]
Get:13 http://security.ubuntu.com/ubun